In [1]:
#remove cell visibility
from IPython.display import HTML
tag = HTML('''<script>
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.input').hide()
    } else {
        $('div.input').show()
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
Promijeni vidljivost <a href="javascript:code_toggle()">ovdje</a>.''')
display(tag)

In [2]:
%matplotlib inline
import control
import numpy
import sympy as sym
from IPython.display import display, Markdown
import ipywidgets as widgets
import matplotlib.pyplot as plt


#print a matrix latex-like
def bmatrix(a):
     """Returns a LaTeX bmatrix - by Damir Arbula (ICCT project)

     :a: numpy array
     :returns: LaTeX bmatrix as a string
     """
     if len(a.shape) > 2:
         raise ValueError('bmatrix can at most display two dimensions')
     lines = str(a).replace('[', '').replace(']', '').splitlines()
     rv = [r'\begin{bmatrix}']
     rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
     rv +=  [r'\end{bmatrix}']
     return '\n'.join(rv)


# Display formatted matrix: 
def vmatrix(a):
    if len(a.shape) > 2:
         raise ValueError('bmatrix can at most display two dimensions')
    lines = str(a).replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{vmatrix}']
    rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
    rv +=  [r'\end{vmatrix}']
    return '\n'.join(rv)


#matrixWidget is a matrix looking widget built with a VBox of HBox(es) that returns a numPy array as value !
class matrixWidget(widgets.VBox):
    def updateM(self,change):
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.M_[irow,icol] = self.children[irow].children[icol].value
                #print(self.M_[irow,icol])
        self.value = self.M_

    def dummychangecallback(self,change):
        pass
    
    
    def __init__(self,n,m):
        self.n = n
        self.m = m
        self.M_ = numpy.matrix(numpy.zeros((self.n,self.m)))
        self.value = self.M_
        widgets.VBox.__init__(self,
                             children = [
                                 widgets.HBox(children = 
                                              [widgets.FloatText(value=0.0, layout=widgets.Layout(width='90px')) for i in range(m)]
                                             ) 
                                 for j in range(n)
                             ])
        
        #fill in widgets and tell interact to call updateM each time a children changes value
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
                self.children[irow].children[icol].observe(self.updateM, names='value')
        #value = Unicode('example@example.com', help="The email value.").tag(sync=True)
        self.observe(self.updateM, names='value', type= 'All')
        
    def setM(self, newM):
        #disable callbacks, change values, and reenable
        self.unobserve(self.updateM, names='value', type= 'All')
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].unobserve(self.updateM, names='value')
        self.M_ = newM
        self.value = self.M_
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].observe(self.updateM, names='value')
        self.observe(self.updateM, names='value', type= 'All')        

                #self.children[irow].children[icol].observe(self.updateM, names='value')

             
#overlaod class for state space systems that DO NOT remove "useless" states (what "professor" of automatic control would do this?)
class sss(control.StateSpace):
    def __init__(self,*args):
        #call base class init constructor
        control.StateSpace.__init__(self,*args)
    #disable function below in base class
    def _remove_useless_states(self):
        pass

## Pneumatsko upravljanje položaja

<img src="Images\EX41.svg" alt="drawing" width="300x300">

Kugla se nalazi unutar vertikalne cijevi na temelju strujanja zraka $u$ i spojena je putem opruge krutosti $K$ na dno cijevi; kugla je podvrgnuta gravitaciji i viskoznom trenju s koeficijentom $B$. Sila $F$ koja na kuglu djeluje protokom zraka proporcionalna je protoku zraka $u$ (u m/s) uz konstantu $G$; protok zraka može biti samo pozitivan (ulazak u cijev). S obzirom na masu kugle $m=2$ g, $B=1$ g/s, $K=2$ g/$\text{s}^2$ i $G=3$ Ns/m, cilj je dizajnirati regulator s ulazom $u$ i visinom položaja kugle $z$ kao izlazom sustava prema sljedećim zahtjevima:
- nulta pogreška u stacionarnom stanju (u odzivu na ulaz željene visine);
- maksimalno prekoračenje 30%
- vrijeme smirivanja za 5% pojasa tolerancije manje od 8 sekundi.

Jednadžba dinamike koja opisuje sustav je:

$$
    m\ddot{z} = -Kz -B\dot{z} + Gu
$$

Definiranjem vektora stanja kao $x=\begin{bmatrix} x_1 & x_2 \end{bmatrix}^T=\begin{bmatrix} z & \dot{z} \end{bmatrix}^T$, možemo pisati: 

\begin{cases}
    \dot{x} = \begin{bmatrix} 0 & 1 \\ -\frac{K}{m} & -\frac{B}{m} \end{bmatrix}x + \begin{bmatrix} 0 \\ \frac{G}{m} \end{bmatrix}u \\
    y = \begin{bmatrix} 1 & 0 \end{bmatrix}x
\end{cases}

Polovi sustava su $-0.25\pm0.97i$.

### Dizajn regulatora
#### Dizajn kontrolera

Da bi se zadovoljila nulta pogreška u stacionarnom stanju, dodajemo integrator u sustav proširivanjem vektora stanja s $\dot{x_3} = x_1 - y_d$, gdje je $y_d$ referentni signal. Tada je prošireni sustav:

\begin{cases}
    \dot{x_a} = \begin{bmatrix} 0 & 1 & 0 \\ -\frac{K}{m} & -\frac{B}{m} & 0 \\ 1 & 0 & 0 \end{bmatrix}x_a + \begin{bmatrix} 0 & 0 \\ \frac{G}{m} & 0 \\ 0 & -1 \end{bmatrix}u \\
    y_a = \begin{bmatrix} 1 & 0 & 0 \\ 0 & 0 & 1 \end{bmatrix}x_a \, .
\end{cases}

Budući da su polovi sustava spori u odnosu na zadane zahtjeve, povećavamo prirodnu frekvenciju kompleksnih polova stavljajući ih u $-0.5\pm1.94$, a, također, integrator zamjenjujemo s polom u $-2$.




#### Dizajn promatrača
Za procjenu stanja, razvijamo promatrač za podsustav $(x_1,\, x_2)$, a zatim uzimamo samo procjenu za $x_2$, jer se ostala stanja mjere izravno. Odabiremo $-5$ kao mjesto polova standardnog promatrača.

### Kako koristiti ovaj interaktivni primjer?
- Provjerite rad regulatora relativnom varijacijom mase kugle (parametar: delta m) i, ako je potrebno, pokušajte ispraviti poziciju pola.
- Testirajte regulator s početnom pogreškom u estimiranom stanju.
- Testirajte regulator s različitim tipovima referentnog ulaza.

In [3]:
# Preparatory cell
b = 1E-03
k = 2E-03
G = 3
m = 2E-03
Aa = numpy.matrix([[0, 1, 0],
                   [-k/m, -b/m, 0],
                   [1, 0, 0]])
Ba = numpy.matrix([[0, 0],
                   [G/m, 0],
                   [0,-1]])
Ca = numpy.matrix([[1, 0, 0],
                   [0, 0, 1]])
A = Aa[0:2,0:2]
B = Ba[0:2,0]
C = Ca[0,0:2]

X0 = numpy.matrix('0.0')
K = numpy.matrix([8/15,-4.4,-4])
L = numpy.matrix([[23],[66]])

X0w = matrixWidget(1,1)
X0w.setM(X0)
Kw = matrixWidget(1,3)
Kw.setM(K)
Lw = matrixWidget(2,1)
Lw.setM(L)


eig1c = matrixWidget(1,1)
eig2c = matrixWidget(2,1)
eig3c = matrixWidget(1,1)
eig1c.setM(numpy.matrix([-2.0])) 
eig2c.setM(numpy.matrix([[-0.5],[-1.94]]))
eig3c.setM(numpy.matrix([-2.0]))

eig1o = matrixWidget(1,1)
eig2o = matrixWidget(2,1)
eig1o.setM(numpy.matrix([-5.])) 
eig2o.setM(numpy.matrix([[-5.],[0.]]))

In [4]:
# Misc

#create dummy widget 
DW = widgets.FloatText(layout=widgets.Layout(width='0px', height='0px'))

#create button widget
START = widgets.Button(
    description='Test',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Test',
    icon='check'
)
                       
def on_start_button_clicked(b):
    #This is a workaround to have intreactive_output call the callback:
    #   force the value of the dummy widget to change
    if DW.value> 0 :
        DW.value = -1
    else: 
        DW.value = 1
    pass
START.on_click(on_start_button_clicked)

# Define type of method 
selm = widgets.Dropdown(
    options= ['Postavi K i L', 'Postavi svojstvene vrijednosti'],
    value= 'Postavi svojstvene vrijednosti',
    description='',
    disabled=False
)

# Define the number of complex eigenvalues
sele = widgets.Dropdown(
    options= ['0 kompleksnih svojstvenih vrijednosti', '2 kompleksne svojstvene vrijednosti'],
    value= '2 kompleksne svojstvene vrijednosti',
    description='Kompleksne svojstvene vrijednosti:',
    disabled=False
)

#define type of ipout 
selu = widgets.Dropdown(
    options=['impuls', 'step', 'sinus', 'Pravokutni val'],
    value='step',
    description='Tip smetnje:',
    style = {'description_width': 'initial'},
    disabled=False
)
# Define the values of the input
u = widgets.FloatSlider(
    value=0.1,
    min=0,
    max=0.8,
    step=0.1,
    description='Referentni signal [m]:',
    style = {'description_width': 'initial'},
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
mw = widgets.FloatSlider(
    value=0,
    min=-30,
    max=30,
    step=1,
    description='delta m [%]:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
period = widgets.FloatSlider(
    value=2,
    min=0.1,
    max=3,
    step=0.05,
    description='Period [s]: ',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)
simTime = widgets.FloatText(
    value=8,
    description='',
    disabled=False
)

In [5]:
# Support functions

def eigen_choice(sele):
    if sele == '0 kompleksnih svojstvenih vrijednosti':
        eig1c.children[0].children[0].disabled = False
        eig2c.children[1].children[0].disabled = True
        eig1o.children[0].children[0].disabled = False
        eig2o.children[1].children[0].disabled = True
        eig = 0
    if sele == '2 kompleksne svojstvene vrijednosti':
        eig1c.children[0].children[0].disabled = True
        eig2c.children[1].children[0].disabled = False
        eig1o.children[0].children[0].disabled = True
        eig2o.children[1].children[0].disabled = False
        eig = 2
    return eig

def method_choice(selm):
    if selm == 'Postavi K i L':
        method = 1
        sele.disabled = True
    if selm == 'Postavi svojstvene vrijednosti':
        method = 2
        sele.disabled = False
    return method

def simulation(m,Kw,L,uw,selu,period,simTime,X0w):
    Aa = numpy.matrix([[0, 1, 0],
                   [-k/(0.002*(1+m/100.0)), -b/(0.002*(1+m/100.0)), 0],
                   [1, 0, 0]])
    Ba = numpy.matrix([[0, 0],
                       [G/(0.002*(1+m/100.0)), 0],
                       [0,-1]])
    A = Aa[0:2,0:2]
    B = Ba[0:2,0]
    
    e1 = numpy.linalg.eig(Aa-Ba[:,0]*Kw)[0]
    e1n = numpy.array([numpy.linalg.norm(e1[i]) for i in range(3)])
    e2 = numpy.linalg.eig(A-L*C)[0]
    e2n = numpy.array([numpy.linalg.norm(e2[i]) for i in range(2)])
    # print(e1n,e2n)
    
    n1 = 0
    if all(e1n > 0):
        if all(numpy.real(e1)<0):
            n1 = int(max(e1n)/2/numpy.pi*1000)
    if n1==0:
        n1 = 10000
    
    n2 = 0
    if all(e2n > 0):
        if all(numpy.real(e2)<0):
            n2 = int(max(e2n)/2/numpy.pi*1000)
    if n2==0:
        n2 = 10000
    if n1 >= n2:
        n = n1
    else:
        n = n2
    
    T = numpy.linspace(0,simTime,n)
    ts = T[1]-T[0]
    if selu == 'impuls': #selu
        R = [0 for t in range(0,len(T))]
        R[0] = uw        
    if selu == 'step':
        R = [uw for t in range(0,len(T))]
    if selu == 'sinus':
        R = uw*numpy.sin(2*numpy.pi/period*T)
        #print(R)
    if selu == 'Pravokutni val':
        R = uw*numpy.sign(numpy.sin(2*numpy.pi/period*T))
        #print(R)
    
    Aad = (numpy.eye(3)+ts*Aa)
    Bad = ts*Ba
    Ad = (numpy.eye(2)+ts*(A-L*C))
    Bd = ts*B
    Ld = ts*L
    xa = numpy.matrix([[0],[0],[0]])
    xe = numpy.matrix([[0],[X0w]])
    x = numpy.vstack((xa,xe))
    X = [x]
    U = []
    
    for i in range(len(T)):
        u = (-Kw*numpy.matrix([[xa[0,0]],[xe[1,0]],[xa[2,0]]]))[0,0]
        if u < 0:
            u = 0
            
        xe = Ad*xe+Bd*u+Ld*xa[0,0]
        xa = Aad*xa+Bad*numpy.matrix([[u],[R[i]]])
        
        X.append(numpy.vstack((xa,xe)))
        U.append(u)
    
    xout = [
        [X[i][0,0] for i in range(len(T))],
        [X[i][1,0] for i in range(len(T))],
        [X[i][2,0] for i in range(len(T))],
        [X[i][3,0] for i in range(len(T))],
        [X[i][4,0] for i in range(len(T))]
    ]
    return T, U, R, xout, e1, e2

In [6]:
def main_callback2(mw, X0w, K, L, eig1c, eig2c, eig3c, eig1o, eig2o, u, period, selm, sele, selu, simTime, DW):
    eige = eigen_choice(sele)
    method = method_choice(selm)
    
    if method == 1:
        solc = numpy.linalg.eig(Aa-Ba[:,0]*K)[0]
        solo = numpy.linalg.eig(A-L*C)[0]
    if method == 2:
        if eige == 0:
            K = control.acker(Aa, Ba[:,0], [eig1c[0,0], eig2c[0,0], eig3c[0,0]])
            Kw.setM(K)
            L = control.acker(A.T, C.T, [eig1o[0,0], eig2o[0,0]]).T
            Lw.setM(L)
        if eige == 2:
            K = control.acker(Aa, Ba[:,0], [eig3c[0,0], 
                                     numpy.complex(eig2c[0,0],eig2c[1,0]), 
                                     numpy.complex(eig2c[0,0],-eig2c[1,0])])
            Kw.setM(K)
            L = control.acker(A.T, C.T, [numpy.complex(eig2o[0,0],eig2o[1,0]), 
                                         numpy.complex(eig2o[0,0],-eig2o[1,0])]).T
            Lw.setM(L)
            
    if simTime != 0:
        pass
    else:
        simTime = 7
        
    T, U, R, xout, sol1, sol2 = simulation(mw,K,L,u,selu,period,simTime,X0w[0,0])
    
    if selu == 'step':
        found = False
        ST5 = 0
        for i in range(len(T)):
            if R[i] != 0:
                if abs((xout[0][i]-R[i])/R[i]) <= 0.05 and not found:
                    ST5 = T[i]
                    found = True
                if abs((xout[0][i]-R[i])/R[i]) > 0.05:
                    found = False
        OV = 0
        for i in range(len(T)):
            if R[i] != 0:
                if R[0]>0:
                    if (xout[0][i]-R[i]) > 0:
                        if abs((xout[0][i]-R[i])/R[i])*100 > OV:
                            OV = abs((xout[0][i]-R[i])/R[i])*100
                else:
                    if (xout[0][i]-R[i]) < 0:
                        if abs((xout[0][i]-R[i])/R[i])*100 > OV:
                            OV = abs((xout[0][i]-R[i])/R[i])*100
    else:
        ST5 = 'Nije definirano' 
        OV = 'Nije definirano'
    print('Svojstvene vrijednosti sustava zatvorene petlje:',sol1)
    print('Svojstvene vrijednosti promatrača:',sol2)
    print('Informacije o koraku: \n\tVrijeme smirivanja (5%) [s]=',ST5,'\n\tPrekoračenje (%)=',OV)
    
    fig = plt.figure(num='Simulation1', figsize=(14,12))
    
    fig.add_subplot(221)
    plt.title('Izlazni odziv')
    plt.ylabel('Izlaz [m]')
    plt.plot(T,xout[0])
    plt.plot(T,R,'g--')
    plt.xlabel('$t$ [s]')
    plt.axvline(x=0,color='black',linewidth=0.8)
    plt.axhline(y=0,color='black',linewidth=0.8)
    plt.legend(['$y$','Reference'])
    plt.grid()
    
    fig.add_subplot(222)
    plt.title('Ulaz')
    plt.ylabel('$u$ [N]')
    plt.plot(T,U)
    plt.xlabel('$t$ [s]')
    plt.axvline(x=0,color='black',linewidth=0.8)
    plt.axhline(y=0,color='black',linewidth=0.8)
    plt.grid()
    
    fig.add_subplot(223)
    plt.title('Odziv stanja')
    plt.ylabel('Stanja')
    plt.plot(T,xout[0],
             T,xout[1],
             T,xout[2])
    plt.xlabel('$t$ [s]')
    plt.axvline(x=0,color='black',linewidth=0.8)
    plt.axhline(y=0,color='black',linewidth=0.8)
    plt.legend(['$x_{1}$','$x_{2}$','$x_{3}$'])
    plt.grid()
    
    fig.add_subplot(224)
    plt.title('Pogreška procjene')
    plt.ylabel('Pogreška')
    plt.plot(T,numpy.array(xout[1])-numpy.array(xout[4]))
    plt.xlabel('$t$ [s]')
    plt.axvline(x=0,color='black',linewidth=0.8)
    plt.axhline(y=0,color='black',linewidth=0.8)
    plt.legend(['$e_{2}$'])
    plt.grid()
    #plt.tight_layout()
   
alltogether2 = widgets.VBox([widgets.HBox([selm, 
                                          sele,
                                          selu]),
                            widgets.Label(' ',border=3),
                            widgets.HBox([widgets.Label('K:',border=3), Kw, 
                                          widgets.Label(' ',border=3),
                                          widgets.Label(' ',border=3),
                                          widgets.Label('Svojstvene vrijednosti:',border=3), 
                                          eig1c, 
                                          eig2c, 
                                          eig3c,
                                          widgets.Label(' ',border=3),
                                          widgets.Label(' ',border=3),
                                          widgets.Label('X0 est.:',border=3), X0w]),
                            widgets.Label(' ',border=3),
                            widgets.HBox([widgets.Label('L:',border=3), Lw, 
                                          widgets.Label(' ',border=3),
                                          widgets.Label(' ',border=3),
                                          widgets.Label('Svojstvene vrijednosti:',border=3), 
                                          eig1o, 
                                          eig2o,
                                          widgets.Label(' ',border=3),
                                          widgets.VBox([widgets.Label('Vrijeme simulacije [s]:',border=3)]),
                                          widgets.VBox([simTime])]),
                            widgets.Label(' ',border=3),
                            widgets.HBox([u,
                                          mw,
                                          period, 
                                          START])])
out2 = widgets.interactive_output(main_callback2, {'mw':mw, 'X0w':X0w, 'K':Kw, 'L':Lw,
                                                 'eig1c':eig1c, 'eig2c':eig2c, 'eig3c':eig3c, 'eig1o':eig1o, 'eig2o':eig2o, 
                                                 'u':u, 'period':period, 'selm':selm, 'sele':sele, 'selu':selu, 'simTime':simTime, 'DW':DW})
out2.layout.height = '860px'
display(out2, alltogether2)

Output(layout=Layout(height='860px'))